In [40]:
import requests
from bs4 import BeautifulSoup


# URL de la page principale
url = "https://www.avito.ma/fr/maroc/voitures_d_occasion-%C3%A0_vendre"


# En-têtes pour éviter le blocage du site
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

# Récupération du contenu HTML de la page principale
response = requests.get(url, headers=headers)

# Vérification du statut de la requête
if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")

    # Trouver tous les liens sur la page principale
    div_principal = soup.find("div", {'class':'sc-1nre5ec-1 crKvIr listing'})
    liens = div_principal.find_all("a", href=True)

    # Filtrer les liens qui commencent par "/fr/" (qui sont des liens relatifs pour les annonces)
    # liens_annonces = [urljoin(url, lien["href"]) for lien in liens if lien["href"].startswith("/fr/")]

    # Afficher les liens extraits
    for lien in liens:
        print(f"Accès à l'annonce : {lien['href']}")
        
        # Récupérer les détails pour chaque annonce
        annonce_response = requests.get(lien["href"], headers=headers)
        
        if annonce_response.status_code == 200:
            annonce_soup = BeautifulSoup(annonce_response.text, "html.parser")
            
            # Extraction des informations du div principal dans l'annonce
            div_annonce = annonce_soup.find("div", class_="sc-1g3sn3w-8 ePtCCn")

            if div_annonce:
                # Extraire le titre dans le <h1>
                titre = div_annonce.find("h1").text.strip() 

                # Extraire la description dans le <p>
                description = div_annonce.find("p").text.strip()
            
                div = annonce_soup.find("div", class_="sc-1g3sn3w-7 bNWHpB")

                # Extraire le texte dans le premier <span>
                span1 = div.find_all("span")[0].text.strip()

                # Extraire le texte dans le deuxième <span>
                span2 = div.find_all("span")[1].text.strip() 

                # Extractions des ifos suplimentaires
                div_infos_sup = annonce_soup.find("div", class_="sc-6p5md9-0 dsWaSi")
                if div_infos_sup:

                    # annee de voiture
                    anne=div_infos_sup.find_all("span")[0].text.strip()
                    # Type du voiture AUTO/MANUEL
                    auto_manuel=div_infos_sup.find_all("span")[1].text.strip()
                    
                    # Type de V Diesl/Hybride
                    diesl_hybr=div_infos_sup.find_all("span")[2].text.strip()
                    
                # Extraction du donnees de voiture
                div_infos_sup_2 = annonce_soup.find("ol", class_="sc-qmn92k-3 hTEaGJ")
                if div_infos_sup_2:
                    elements_liste = div_infos_sup_2.find_all("li")
                    type = elements_liste[0].find_all("span")[1].text.strip()
                    secteur = elements_liste[1].find_all("span")[1].text.strip()
                    kilometrage = elements_liste[2].find_all("span")[1].text.strip()
                    marque = elements_liste[3].find_all("span")[1].text.strip()
                    modele = elements_liste[4].find_all("span")[1].text.strip()
                    n_portes = elements_liste[5].find_all("span")[1].text.strip()
                    origine = elements_liste[6].find_all("span")[1].text.strip()
                    p_main = elements_liste[7].find_all("span")[1].text.strip()
                    p_fiscale = elements_liste[8].find_all("span")[1].text.strip()
                    etat = elements_liste[9].find_all("span")[1].text.strip()
                    
            # Afficher les informations extraites
                print(f"Titre : {titre}")
                print(f"Description : {description}")
                print(f"Date d'ajout : {span2}")
                print(f"Localisation : {span1}")
                print(f"la anne : {anne}")
                print(f"auto ou manul : {auto_manuel}")
                print(f"type disel ou hyb : {diesl_hybr}")
                print(f"Type : {type}")
                print(f"Secteur: {secteur}")
                print(f"Kilometrage : {kilometrage}")
                print(f"Marque : {marque}")
                print(f"Modele : {modele}")
                print(f"Nombre de portes: {n_portes}")
                print(f"Origine : {origine}")
                print(f"Premier main : {p_main}")
                print(f"Puissance fiscale : {p_fiscale}")
                print(f"Etat : {etat}")
                print("="*50)    
                
            else:
                print(f"Div principal non trouvé pour l'annonce : {lien}")
      
        else:
            print(f"Erreur de chargement pour l'annonce : {lien}")
            
else:
    print("Erreur de chargement de la page principale :", response.status_code)

KeyboardInterrupt: 

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import os

# Chemin du fichier CSV
csv_file_path = r"C:\Users\HP\Desktop\projet ai\mini projet 2\avito_voitures - Copy - Copy.csv"

# En-têtes CSV
csv_headers = [
    "Titre", "Prix", "Date d'ajout", "Localisation","Annee","Auto_manuel","Carburant", "Type", "Secteur", 
    "Kilométrage", "Marque", "Modèle", "Nombre de portes", "Origine", 
    "Première main", "Puissance fiscale", "État", 
]

# URL de base
base_url = "https://www.avito.ma/fr/casablanca/voitures_d_occasion-%C3%A0_vendre"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

def get_page_content(url):
    try:
        response = requests.get(url, headers=headers, allow_redirects=True)
        if response.status_code == 200:
            return response.text
        else:
            print(f"❌ Erreur {response.status_code} sur : {url}")
    except requests.exceptions.RequestException as e:
        print(f"❌ Exception pour {url} : {e}")
    return None

# Initialiser le CSV
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(csv_headers)  # Écrire les en-têtes

    # Scraper les pages
    for page in range(41, 2):
        url = f"{base_url}?o={page}"
        print(f"\n🔎 Scraping de la page {page}: {url}")
        page_content = get_page_content(url)
        if not page_content:
            continue

        soup = BeautifulSoup(page_content, "html.parser")
        div_principal = soup.find("div", class_='sc-1nre5ec-1 crKvIr listing')
        if not div_principal:
            print(f"⚠️ Pas d'annonces trouvées sur la page {page}")
            continue

        annonces = div_principal.find_all("a", href=True)
        if not annonces:
            print(f"⚠️ Aucune annonce trouvée sur la page {page}")
            continue

        for annonce in annonces:
            annonce_url = annonce["href"]
            if annonce_url.startswith("/"):
                annonce_url = "https://www.avito.ma" + annonce_url

            print(f"📌 Traitement de l'annonce : {annonce_url}")
            annonce_content = get_page_content(annonce_url)
            if not annonce_content:
                continue

            annonce_soup = BeautifulSoup(annonce_content, "html.parser")
            div_annonce = annonce_soup.find("div", class_="sc-1g3sn3w-8 ePtCCn")

            if not div_annonce:
                print(f"⚠️ Aucune donnée trouvée pour l'annonce {annonce_url}")
                continue

            # Extraction des infos principales
            titre = div_annonce.find("h1").text.strip() if div_annonce.find("h1") else "N/A"
            prix = div_annonce.find("p").text.strip() if div_annonce.find("p") else "N/A"
            
            div = annonce_soup.find("div", class_="sc-1g3sn3w-7 bNWHpB")
            date_ajout = div.find_all("span")[1].text.strip() if div else "N/A"
            localisation = div.find_all("span")[0].text.strip() if div else "N/A"

            div_infos_sup = annonce_soup.find("div", class_="sc-6p5md9-0 dsWaSi")
            Annee = div_infos_sup.find_all("span")[0].text.strip() if div_infos_sup else "N/A"
            Auto_manuel = div_infos_sup.find_all("span")[1].text.strip() if div_infos_sup else "N/A"
            Carburant = div_infos_sup.find_all("span")[2].text.strip() if div_infos_sup else "N/A"

            # Infos supplémentaires
            div_infos_sup_2 = annonce_soup.find("ol", class_="sc-qmn92k-3 hTEaGJ")
            try:
                elements = div_infos_sup_2.find_all("li") if div_infos_sup_2 else []
                type_voiture = elements[0].find_all("span")[1].text.strip() if len(elements) > 0 else "N/A"
                secteur = elements[1].find_all("span")[1].text.strip() if len(elements) > 1 else "N/A"
                kilometrage = elements[2].find_all("span")[1].text.strip() if len(elements) > 2 else "N/A"
                marque = elements[3].find_all("span")[1].text.strip() if len(elements) > 3 else "N/A"
                modele = elements[4].find_all("span")[1].text.strip() if len(elements) > 4 else "N/A"
                n_portes = elements[5].find_all("span")[1].text.strip() if len(elements) > 5 else "N/A"
                origine = elements[6].find_all("span")[1].text.strip() if len(elements) > 6 else "N/A"
                p_main = elements[7].find_all("span")[1].text.strip() if len(elements) > 7 else "N/A"
                p_fiscale = elements[8].find_all("span")[1].text.strip() if len(elements) > 8 else "N/A"
                etat = elements[9].find_all("span")[1].text.strip() if len(elements) > 9 else "N/A"
            except Exception as e:
                print(f"❌ Erreur extraction détails : {e}")
                type_voiture = secteur = kilometrage = marque = modele = n_portes = origine = p_main = p_fiscale = etat = "N/A"

            # Stocker les données dans le CSV
            writer.writerow([
            titre,prix, date_ajout, localisation,Annee,Auto_manuel,Carburant, type_voiture, secteur, 
             kilometrage, marque, modele, n_portes, origine, p_main, 
             p_fiscale, etat,
])

            # Pause entre chaque annonce
            time.sleep(4)

        # Pause entre les pages
        time.sleep(5)

print("\n✅ Scraping terminé et données enregistrées dans le fichier CSV !")



✅ Scraping terminé et données enregistrées dans le fichier CSV !


In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import os

# Chemin du fichier CSV
csv_file_path = r"C:\Users\HP\Desktop\projet ai\mini projet 2\avito_voitures - Copy - Copy - Copy.csv"

# En-têtes CSV
csv_headers = [
    "Titre", "Prix", "Date d'ajout", "Localisation", "Annee", "Auto_manuel", "Carburant",
    "Type", "Secteur", "Kilométrage", "Marque", "Modèle", "Nombre de portes",
    "Origine", "Première main", "Puissance fiscale", "État",
]

# URL de base
base_url = "https://www.avito.ma/fr/casablanca/voitures_d_occasion-%C3%A0_vendre"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

# Fichier pour suivre la dernière page scrappée
last_scraped_page_file = "last_scraped_page.txt"

def get_page_content(url):
    try:
        response = requests.get(url, headers=headers, allow_redirects=True)
        if response.status_code == 200:
            return response.text
        else:
            print(f"❌ Erreur {response.status_code} sur : {url}")
    except requests.exceptions.RequestException as e:
        print(f"❌ Exception pour {url} : {e}")
    return None

def get_last_scraped_page():
    if os.path.exists(last_scraped_page_file):
        with open(last_scraped_page_file, "r") as f:
            return int(f.read())
    return 1  # Commencer par la première page si le fichier n'existe pas

def save_last_scraped_page(page):
    with open(last_scraped_page_file, "w") as f:
        f.write(str(page))

def clean_text(text):
    if text:
        return text.encode('utf-8').decode('utf-8').strip()
    return None

# Initialisation du CSV : si vide, écrire les headers
if not os.path.exists(csv_file_path) or os.stat(csv_file_path).st_size == 0:
    with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(csv_headers)

# Scraper en mode ajout (append)
with open(csv_file_path, mode='a', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)

    start_page = get_last_scraped_page()
    pages_to_scrape = 1 # Tu peux modifier

    for page in range(start_page, start_page + pages_to_scrape):
        url = f"{base_url}?o={page}"
        print(f"\n🔎 Scraping de la page {page}: {url}")
        page_content = get_page_content(url)
        if not page_content:
            continue

        soup = BeautifulSoup(page_content, "html.parser")
        div_principal = soup.find("div", class_='sc-1nre5ec-1 crKvIr listing')
        if not div_principal:
            print(f"⚠️ Pas d'annonces trouvées sur la page {page}")
            continue

        annonces = div_principal.find_all("a", href=True)
        if not annonces:
            print(f"⚠️ Aucune annonce trouvée sur la page {page}")
            continue

        for annonce in annonces:
            annonce_url = annonce["href"]
            if annonce_url.startswith("/"):
                annonce_url = "https://www.avito.ma" + annonce_url

            print(f"📌 Traitement de l'annonce : {annonce_url}")
            annonce_content = get_page_content(annonce_url)
            if not annonce_content:
                continue

            annonce_soup = BeautifulSoup(annonce_content, "html.parser")
            div_annonce = annonce_soup.find("div", class_="sc-1g3sn3w-8 ePtCCn")
            if not div_annonce:
                print(f"⚠️ Aucune donnée trouvée pour l'annonce {annonce_url}")
                continue

            titre = div_annonce.find("h1").text.strip() if div_annonce.find("h1") else None
            prix = div_annonce.find("p").text.strip() if div_annonce.find("p") else None
            prix = None if prix and "Prix non spécifié" in prix else prix

            div = annonce_soup.find("div", class_="sc-1g3sn3w-7 bNWHpB")
            date_ajout = div.find_all("span")[1].text.strip() if div else None
            localisation = div.find_all("span")[0].text.strip() if div else None

            div_infos_sup = annonce_soup.find("div", class_="sc-6p5md9-0 dsWaSi")
            Annee = div_infos_sup.find_all("span")[0].text.strip() if div_infos_sup else None
            Auto_manuel = div_infos_sup.find_all("span")[1].text.strip() if div_infos_sup else None
            Carburant = div_infos_sup.find_all("span")[2].text.strip() if div_infos_sup else None

            div_infos_sup_2 = annonce_soup.find("ol", class_="sc-qmn92k-3 hTEaGJ")
            try:
                elements = div_infos_sup_2.find_all("li") if div_infos_sup_2 else []
                type_voiture = elements[0].find_all("span")[1].text.strip() if len(elements) > 0 else None
                secteur = elements[1].find_all("span")[1].text.strip() if len(elements) > 1 else None
                kilometrage = elements[2].find_all("span")[1].text.strip() if len(elements) > 2 else None
                marque = elements[3].find_all("span")[1].text.strip() if len(elements) > 3 else None
                modele = elements[4].find_all("span")[1].text.strip() if len(elements) > 4 else None
                n_portes = elements[5].find_all("span")[1].text.strip() if len(elements) > 5 else None
                origine = elements[6].find_all("span")[1].text.strip() if len(elements) > 6 else None
                p_main = elements[7].find_all("span")[1].text.strip() if len(elements) > 7 else None
                p_fiscale = elements[8].find_all("span")[1].text.strip() if len(elements) > 8 else None
                etat = elements[9].find_all("span")[1].text.strip() if len(elements) > 9 else None
            except Exception as e:
                print(f"❌ Erreur extraction détails : {e}")
                type_voiture = secteur = kilometrage = marque = modele = n_portes = origine = p_main = p_fiscale = etat = None

            # Enregistrer l'annonce dans le CSV
            writer.writerow([
                clean_text(titre),
                clean_text(prix),
                clean_text(date_ajout),
                clean_text(localisation),
                clean_text(Annee),
                clean_text(Auto_manuel),
                clean_text(Carburant),
                clean_text(type_voiture),
                clean_text(secteur),
                clean_text(kilometrage),
                clean_text(marque),
                clean_text(modele),
                clean_text(n_portes),
                clean_text(origine),
                clean_text(p_main),
                clean_text(p_fiscale),
                clean_text(etat),
            ])

            # Pause entre les annonces
            time.sleep(4)

        # Sauvegarde de la dernière page
        save_last_scraped_page(page)

        # Pause entre les pages
        time.sleep(5)

print("\n✅ Scraping terminé et données enregistrées dans le fichier CSV sans perte !")



🔎 Scraping de la page 41: https://www.avito.ma/fr/casablanca/voitures_d_occasion-%C3%A0_vendre?o=41
📌 Traitement de l'annonce : https://www.avito.ma/fr/aïn_chock/voitures_d'occasion/Bmw_x5_très_bon_etat__55998716.htm
📌 Traitement de l'annonce : https://www.avito.ma/fr/anfa/voitures_d'occasion/Pajero_sport_full_option__55925987.htm
📌 Traitement de l'annonce : https://www.avito.ma/fr/ain_sebaa/voitures_d'occasion/Dacia_sandero_Diesel_manuelle_2024_à_Casablanca_56112194.htm
📌 Traitement de l'annonce : https://www.avito.ma/fr/aïn_chock/voitures_d'occasion/_Porsche___Macan_S_55958130.htm
📌 Traitement de l'annonce : https://www.avito.ma/fr/californie/voitures_d'occasion/Mercedes_Benz_Classe_GLE_300_D_Automatique_2020_56223383.htm
📌 Traitement de l'annonce : https://www.avito.ma/fr/aïn_chock/voitures_d'occasion/Dacia_sandero_steepway_diesel_première_main__56013563.htm
📌 Traitement de l'annonce : https://www.avito.ma/fr/sidi_moumen/voitures_d'occasion/Dacia_Logan__55844908.htm
📌 Traitement de

#--------------------------------

In [2]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import os
import re

# Chemin du fichier CSV
csv_file_path = r"C:\Users\HP\Desktop\projet ai\mini projet 2\avito_voitures.csv"

# En-têtes CSV
csv_headers = [
    "Titre", "Prix", "Date d'ajout", "Localisation", "Annee", "Auto_manuel", "Carburant",
    "Type", "Secteur", "Kilométrage", "Marque", "Modèle", "Nombre de portes",
    "Origine", "Première main", "Puissance fiscale", "État",
]

base_url = "https://www.avito.ma/fr/casablanca/voitures_d_occasion-%C3%A0_vendre"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

last_scraped_page_file = "last_scraped_page.txt"

def get_page_content(url):
    try:
        response = requests.get(url, headers=headers, allow_redirects=True)
        if response.status_code == 200:
            response.encoding = 'utf-8'  # Force encoding to UTF-8
            return response.text
        else:
            print(f"❌ Erreur {response.status_code} sur : {url}")
    except requests.exceptions.RequestException as e:
        print(f"❌ Exception pour {url} : {e}")
    return None

def get_last_scraped_page():
    if os.path.exists(last_scraped_page_file):
        with open(last_scraped_page_file, "r") as f:
            return int(f.read())
    return 1

def save_last_scraped_page(page):
    with open(last_scraped_page_file, "w") as f:
        f.write(str(page))

def clean_text(text):
    if text:
        # Supprimer les espaces insécables, les retours à la ligne, les tabulations et espaces superflus
        cleaned = text.replace('\u202f', ' ').replace('\xa0', ' ').replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
        cleaned = re.sub(' +', ' ', cleaned)  # Remplace plusieurs espaces par un seul
        return cleaned.strip()
    return None

# Initialisation du CSV : si vide, écrire les headers
if not os.path.exists(csv_file_path) or os.stat(csv_file_path).st_size == 0:
    with open(csv_file_path, mode='w', newline='', encoding='utf-8-sig') as csvfile:  # Use utf-8-sig for proper encoding in CSV
        writer = csv.writer(csvfile)
        writer.writerow(csv_headers)

# Scraping et sauvegarde continue
with open(csv_file_path, mode='a', newline='', encoding='utf-8-sig') as csvfile:  # Use utf-8-sig for proper encoding in CSV
    writer = csv.writer(csvfile)
    start_page = get_last_scraped_page()
    pages_to_scrape = 340  # Modifier selon le besoin

    for page in range(start_page, start_page + pages_to_scrape):
        url = f"{base_url}?o={page}"
        print(f"\n🔎 Scraping de la page {page}: {url}")
        page_content = get_page_content(url)
        if not page_content:
            continue

        soup = BeautifulSoup(page_content, "html.parser")  # Removed from_encoding='utf-8'
        div_principal = soup.find("div", class_='sc-1nre5ec-1 crKvIr listing')
        if not div_principal:
            print(f"⚠️ Pas d'annonces trouvées sur la page {page}")
            continue

        annonces = div_principal.find_all("a", href=True)
        if not annonces:
            print(f"⚠️ Aucune annonce trouvée sur la page {page}")
            continue

        for annonce in annonces:
            annonce_url = annonce["href"]
            if annonce_url.startswith("/"):
                annonce_url = "https://www.avito.ma" + annonce_url

            print(f"📌 Traitement de l'annonce : {annonce_url}")
            annonce_content = get_page_content(annonce_url)
            if not annonce_content:
                continue

            annonce_soup = BeautifulSoup(annonce_content, "html.parser")  # Removed from_encoding='utf-8'
            div_annonce = annonce_soup.find("div", class_="sc-1g3sn3w-8 ePtCCn")
            if not div_annonce:
                print(f"⚠️ Aucune donnée trouvée pour l'annonce {annonce_url}")
                continue

            titre = div_annonce.find("h1").text if div_annonce.find("h1") else None
            prix = div_annonce.find("p").text if div_annonce.find("p") else None
            prix = None if prix and "Prix non spécifié" in prix else prix

            div = annonce_soup.find("div", class_="sc-1g3sn3w-7 bNWHpB")
            date_ajout = div.find_all("span")[1].text if div else None
            localisation = div.find_all("span")[0].text if div else None

            div_infos_sup = annonce_soup.find("div", class_="sc-6p5md9-0 dsWaSi")
            Annee = div_infos_sup.find_all("span")[0].text if div_infos_sup else None
            Auto_manuel = div_infos_sup.find_all("span")[1].text if div_infos_sup else None
            Carburant = div_infos_sup.find_all("span")[2].text if div_infos_sup else None

            div_infos_sup_2 = annonce_soup.find("ol", class_="sc-qmn92k-3 hTEaGJ")
            try:
                elements = div_infos_sup_2.find_all("li") if div_infos_sup_2 else []
                type_voiture = elements[0].find_all("span")[1].text if len(elements) > 0 else None
                secteur = elements[1].find_all("span")[1].text if len(elements) > 1 else None
                kilometrage = elements[2].find_all("span")[1].text if len(elements) > 2 else None
                marque = elements[3].find_all("span")[1].text if len(elements) > 3 else None
                modele = elements[4].find_all("span")[1].text if len(elements) > 4 else None
                n_portes = elements[5].find_all("span")[1].text if len(elements) > 5 else None
                origine = elements[6].find_all("span")[1].text if len(elements) > 6 else None
                p_main = elements[7].find_all("span")[1].text if len(elements) > 7 else None
                p_fiscale = elements[8].find_all("span")[1].text if len(elements) > 8 else None
                etat = elements[9].find_all("span")[1].text if len(elements) > 9 else None
            except Exception as e:
                print(f"❌ Erreur extraction détails : {e}")
                type_voiture = secteur = kilometrage = marque = modele = n_portes = origine = p_main = p_fiscale = etat = None

            # Enregistrement de l'annonce dans le CSV avec nettoyage
            writer.writerow([
                clean_text(titre),
                clean_text(prix),
                clean_text(date_ajout),
                clean_text(localisation),
                clean_text(Annee),
                clean_text(Auto_manuel),
                clean_text(Carburant),
                clean_text(type_voiture),
                clean_text(secteur),
                clean_text(kilometrage),
                clean_text(marque),
                clean_text(modele),
                clean_text(n_portes),
                clean_text(origine),
                clean_text(p_main),
                clean_text(p_fiscale),
                clean_text(etat),
            ])
            csvfile.flush()  # Pour éviter la perte des données en cas de crash
            time.sleep(4)  # Pause entre chaque annonce

        save_last_scraped_page(page)  # Sauvegarde de la page actuelle
        time.sleep(5)  # Pause entre les pages

print("\n✅ Scraping terminé et toutes les données sont nettoyées et sauvegardées correctement dans le CSV !")



🔎 Scraping de la page 1: https://www.avito.ma/fr/casablanca/voitures_d_occasion-%C3%A0_vendre?o=1
📌 Traitement de l'annonce : https://www.avito.ma/fr/2_mars/voitures_d'occasion/Megane_3_coupé_56153271.htm
📌 Traitement de l'annonce : https://www.avito.ma/fr/bourgogne/voitures_d'occasion/Nissan_Juke_Automatique_2020_à_Casablanca_55599638.htm
📌 Traitement de l'annonce : https://www.avito.ma/fr/californie/voitures_d'occasion/Jeep_Wrangler_Sahara_Unlimited__56229993.htm
📌 Traitement de l'annonce : https://www.avito.ma/fr/belvédère/voitures_d'occasion/Toyota_Corolla_Hybride_Automatique_2020_56229898.htm
📌 Traitement de l'annonce : https://www.avito.ma/fr/2_mars/voitures_d'occasion/Renault_arkana_full_2023_53878936.htm
📌 Traitement de l'annonce : https://www.avito.ma/fr/2_mars/voitures_d'occasion/DACIA_SANDERO_56178015.htm
📌 Traitement de l'annonce : https://www.avito.ma/fr/autre_secteur/voitures_d'occasion/HYUNDAI_TUCSON_Diesel_Automatique_à_Casablanca__55875887.htm
📌 Traitement de l'annonc

KeyboardInterrupt: 